In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# set log level to debug
import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [3]:
from task import VolcengineMLTaskManager

In [4]:
volcengine_task_manager = VolcengineMLTaskManager('config/volc_mltask.yaml')

In [23]:
task_id1 = volcengine_task_manager.submit(
    'test_task_1',
    'nvidia-smi; ls /vepfs-mlp/hongning.zhang -hla; exit 1'
)
task_id2 = volcengine_task_manager.submit(
    'test_task_2',
    'nvidia-smi; ls /vepfs-mlp/hongning.zhang -hla; echo ok',
    dependencies={
        'afternotok': task_id1
    }
)
volcengine_task_manager.wait(task_id1)

2024-02-26 19:28:58,328 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): open.volcengineapi.com:80
2024-02-26 19:29:00,422 - urllib3.connectionpool - DEBUG - http://open.volcengineapi.com:80 "POST /?Action=CreateCustomTask&Version=2021-10-01 HTTP/1.1" 200 None
2024-02-26 19:29:00,523 - urllib3.connectionpool - DEBUG - http://open.volcengineapi.com:80 "POST /?Action=GetCustomTask&Version=2021-10-01 HTTP/1.1" 200 None
2024-02-26 19:29:00,659 - urllib3.connectionpool - DEBUG - http://open.volcengineapi.com:80 "POST /?Action=GetCustomTask&Version=2021-10-01 HTTP/1.1" 200 None
2024-02-26 19:29:00,662 - task.base - DEBUG - Task 20240226-192858-8171 is TaskStatus.PENDING, waiting for afternotok to submit task 20240226-192858-5277
2024-02-26 19:29:01,577 - urllib3.connectionpool - DEBUG - http://open.volcengineapi.com:80 "POST /?Action=GetCustomTask&Version=2021-10-01 HTTP/1.1" 200 None
2024-02-26 19:29:01,722 - urllib3.connectionpool - DEBUG - http://open.volcengineapi.com

<TaskStatus.FAILED: 4>

In [8]:
from task.base import TaskStatus

task_id1 = volcengine_task_manager.submit(
    name="test_depes_afterok_1",
    entrypoint_path="nvidia-smi; sleep 5; echo ok",
)
task_id2 = volcengine_task_manager.submit(
    name="test_depes_afterok_2",
    entrypoint_path="nvidia-smi; sleep 1; echo ok",
    dependencies={"afterok": task_id1}
)
assert task_id1 is not None
assert task_id2 is not None
assert volcengine_task_manager.wait(
    task_id1, waiting_status=(TaskStatus.PENDING,)) == TaskStatus.RUNNING

In [9]:
assert volcengine_task_manager.status(task_id2) == TaskStatus.PENDING
assert volcengine_task_manager.wait(task_id2) == TaskStatus.COMPLETED

In [3]:
from task.slurm import SlurmTaskManager
slurm_task_manager = SlurmTaskManager('config/slurm_task.yaml')

In [60]:
jobid=slurm_task_manager.submit(
    'test_task',
    'nvidia-smi; sleep 80; exit 1',
    partitions='gpu',
)

In [65]:
jobid=slurm_task_manager.submit(
    'test_task',
    'nvidia-smi',
    partitions='gpu',
    dependencies={
        'after': 42670
    }
)

In [66]:
slurm_task_manager.list()

,name,entrypoint_path,status
task_id,,,
42670,test_task,nvidia-smi; sleep 80; exit 1,TaskStatus.FAILED
42671,test_task,nvidia-smi,TaskStatus.PENDING
42672,test_task,nvidia-smi,TaskStatus.COMPLETED


In [68]:
slurm_task_manager.cancel(42671)

In [ ]:
slurm_task_manager.cancel(jobid)
slurm_task_manager.list()
slurm_task_manager.wait(jobid)

In [6]:
task_id = slurm_task_manager.submit(
            name="test_cancel_task",
            entrypoint_path="nvidia-smi; sleep 1; echo notok")
print(task_id)
!/data/slurm/bin/squeue -u hongning.zhang

42691
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             42691       gpu test_can hongning PD       0:00      1 (None)
